In [1]:
from database_io import *
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers


In [2]:
# path of databases (must exist)
db_path = "db/"

# filenames of databases (this must be sqlite3 databases)
train_fname = "surf17_train.db"
validation_fname = "surf17_validation.db"
test_fname = "surf17_test.db"

data = DatabaseIO(dim_syndr=8, dim_fsyndr=4)

In [3]:
try:
    data.close_databases()
except:
    pass
data.load_data(db_path + train_fname, db_path + validation_fname, db_path + test_fname)



batch_size = 10
n_batches_train = 100
n_batches_validation = 100

def train_gen():
    for batch_x1, batch_x2, batch_fx, batch_l1, batch_l2, batch_y in data.gen_batches(batch_size, n_batches_train, data_type='training'):
        yield (batch_x1, batch_y)

def valid_gen():
    for batch_x1, batch_x2, batch_fx, batch_l1, batch_l2, batch_y in data.gen_batches(batch_size, n_batches_train, data_type='validation'):
        yield (batch_x1, batch_y)

# train_data = tf.data.Dataset.from_generator(data.gen_batches)
# valid_data = tf.data.Dataset.from_generator(valid_gen)

loaded databases and checked exclusiveness training, validation, and test keys
N_training=40000, N_validaiton=1000, N_test=5000.


In [ ]:
# based on code from hands-on 5

model = keras.models.Sequential(name="sequential_1")
model.add(layers.LSTM(32, return_sequences=True, name="lstm_1"))
model.add(layers.LSTM(64, return_sequences=True, name="lstm_2"))
model.add(layers.Flatten(name="flatten_1"))
model.add(layers.Dropout(0.3, name="dropout_1"))
model.add(layers.Dense(1, activation="sigmoid", name="dense_1"))

model.summary()

lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-3, decay_steps=100, decay_rate=0.8)

model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=lr_schedule), metrics=["accuracy"])

results = model.fit(
    train_gen(),
    batch_size=batch_size,
    epochs=n_batches_train,
    verbose=1,
    validation_data=valid_gen(),
    callbacks=[
        keras.callbacks.EarlyStopping(patience=15, verbose=0, restore_best_weights=True),
    ],
)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
     97/Unknown 4s 9ms/step - accuracy: 0.5591 - loss: 0.6912

c:\Users\jacky\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\epoch_iterator.py:164: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_1" is incompatible with the layer: expected axis -1 of input shape to have value 1280, but received input with shape (None, 6400)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 100, 8), dtype=bool)
  • training=False
  • mask=None
  • kwargs=<class 'inspect._empty'>